In [46]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Checkbox, VBox,HBox,Label, interact,Box



In [19]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

dataset = helpers.load_adult_income_dataset()

In [2]:
feature_names=['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'gender', 'hours_per_week']


In [7]:
from ipywidgets import interact_manual
from IPython.display import display
#from ipywidgets import TwoByTwoLayout
from ipywidgets import HBox, Label

feature_children=[]
for i in feature_names:
    new_check1=Checkbox(description=i, value=True)
    feature_children.append(new_check1)


feature_vb = VBox(children = feature_children)
use_all_feat = Checkbox(description='1. Use all features? ',value=True)

def add_checks(button1):
    if button1['old']:
        feature_vb.children = feature_children
        print('Deselect features you do not want to vary')
        display(feature_vb)
    else:
        feature_vb.children=[]


use_all_feat.observe(add_checks, names='value')
display(use_all_feat)


def f(a,b,c,d,e,f,g,h):
    use=[a,b,c,d,e,f,g,h]
    using=['age','workclass','education','marital_status','occupation','race','gender','hours_per_week']
    #print(use)
    for i in range(len(use)):
        #print(use[i])
        if use[i] is False:
            using.remove(feature_names[i])
            
    #print(using)
    
    feat_child=[]
    for i in using:
        new_slider=widgets.FloatSlider(description=i, value=1, max=5)
        feat_child.append(new_slider)
        
    vb = VBox(children = feat_child)
    top_toggle1 = Checkbox(description='2. Assign Feature Weights?')

    def add_sliders(button):
        if button['new']:
            vb.children = feat_child
            display(vb)
        else:
            vb.children=[]
    
    top_toggle1.observe(add_sliders, names='value')
    display(top_toggle1)
    

    #display(TwoByTwoLayout(top_left=use_all_feat,top_right=top_toggle1))
    #display(HBox([use_all_feat,top_toggle1]))


out=widgets.interactive_output(f,{'a':feature_children[0],'b':feature_children[1],'c':feature_children[2],'d':feature_children[3],'e':feature_children[4],'f':feature_children[5],'g':feature_children[6],'h':feature_children[7]})
display(out)
















#HIGHLIGHT CHANGEs
highlight_changes=widgets.Checkbox(
    #value=False,
    description='5. Only highlight changes in explanations',
    disabled=False,
    indent=False)

#NUMBER OF EXPLANATIONS
#num_exp=HBox([Label('4. Number of Explanations'), widgets.IntSlider(min=1, max=5)])
num_exp = widgets.IntSlider(description='4. # of Explanations', min=1, max=5,style = {'description_width': 'initial'}, indent=False)
         
# PROX/DIV WEIGHTS
prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5)#default for Dice prox is 0.5
div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1)#default for Dice div is 1


vb_proxdiv = VBox(children = [prox, div])
tune_proxdiv = Checkbox(description='3. Tune proximity/diversity?', indent=False)

def add_2(button):
    if button['new']:
        vb_proxdiv.children = [prox, div]
        display(vb_proxdiv)
    else:
        vb_proxdiv.children=[]
        #TwoByTwoLayout(top_left=tune_proxdiv,
         #      top_right=num_exp,
          #     bottom_right=highlight_changes)

tune_proxdiv.observe(add_2, names='value')




#displaying them
#from ipywidgets import TwoByTwoLayout
#TwoByTwoLayout(top_left=tune_proxdiv,
 #              top_right=num_exp,
  #             bottom_right=highlight_changes)
display(tune_proxdiv, num_exp, highlight_changes)


Checkbox(value=True, description='1. Use all features? ')

Output()

Checkbox(value=False, description='3. Tune proximity/diversity?', indent=False)

IntSlider(value=1, description='4. # of Explanations', max=5, min=1, style=SliderStyle(description_width='init…

Checkbox(value=False, description='5. Only highlight changes in explanations', indent=False)

Deselect features you do not want to vary


In [17]:
cont_feat=['age','hours_per_week']

In [53]:
from ipywidgets import interact_manual
from IPython.display import display
#from ipywidgets import TwoByTwoLayout
from ipywidgets import HBox, Label

print('1. Select Features to use                      2. Feature Weights                    3. Query')

feature_children=[]
for i in feature_names:
    new_check1=Checkbox(description=i, value=True)
    feature_children.append(new_check1)


feature_vb = VBox(children = feature_children)


#display(feature_vb)


def f(a,b,c,d,e,f,g,h):
    use=[a,b,c,d,e,f,g,h]
    using=['age','workclass','education','marital_status','occupation','race','gender','hours_per_week']
    #print(use)
    for i in range(len(use)):
        #print(use[i])
        if use[i] is False:
            using.remove(feature_names[i])
            
    #print(using)
    
    feat_child=[]
    for i in using:
        new_slider=widgets.FloatSlider(description=i, value=1, max=5)
        feat_child.append(new_slider)
        
    vb = VBox(children = feat_child)
    #display(vb)
    #display(HBox([feature_vb,vb]))#,inputq[0]
    

    #display(TwoByTwoLayout(top_left=use_all_feat,top_right=top_toggle1))
    #display(HBox([use_all_feat,top_toggle1]))
    
    
    
    inputq=[]
    #print('Input Query:')

    index=-1
    for i in using:
        index+=1
        if i in cont_feat:
            cont_text=widgets.Text(value='', description=i)
            #display(cont_text)
            inputq.append(cont_text)
        
        else:
            noncont_dropdown=widgets.Dropdown(description=i,options=dataset[i].unique())
            #display(noncont_dropdown)
            inputq.append(noncont_dropdown)
            
    new_children1=[]
    for i in inputq:
        new_children1.append(i)
        
    new_children=VBox(children=new_children1)
        
    display(HBox([feature_vb,vb,new_children]))
    #,inputq[0]
            
            
    


out=widgets.interactive_output(f,{'a':feature_children[0],'b':feature_children[1],'c':feature_children[2],'d':feature_children[3],'e':feature_children[4],'f':feature_children[5],'g':feature_children[6],'h':feature_children[7]})
display(out)
















#HIGHLIGHT CHANGEs
highlight_changes=widgets.Checkbox(
    #value=False,
    description='6. Only highlight changes in explanations',
    disabled=False,
    indent=False)

#NUMBER OF EXPLANATIONS
#num_exp=HBox([Label('4. Number of Explanations'), widgets.IntSlider(min=1, max=5)])
num_exp = widgets.IntSlider(description='5. # of Explanations', min=1, max=5,style = {'description_width': 'initial'}, indent=False)
         
# PROX/DIV WEIGHTS
prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5)#default for Dice prox is 0.5
div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1)#default for Dice div is 1


vb_proxdiv = VBox(children = [prox, div])
tune_proxdiv = Checkbox(description='4. Tune proximity/diversity?', indent=False)

def add_2(button):
    if button['new']:
        vb_proxdiv.children = [prox, div]
        display(vb_proxdiv)
    else:
        vb_proxdiv.children=[]
        #TwoByTwoLayout(top_left=tune_proxdiv,
         #      top_right=num_exp,
          #     bottom_right=highlight_changes)

tune_proxdiv.observe(add_2, names='value')

display(HBox([tune_proxdiv, num_exp, highlight_changes]))



1. Select Features to use                      2. Feature Weights                    3. Query


Output()